# 어울리는 이미지 요소를 추천한 이유에 대해 평가하기
---

## 실험 환경
- 이 노트북은 SageMaker Studio Code Editor 및 커널 base (Python 3.10.13) 에서 테스트 되었습니다.
- 실행 환경에 설치된 Python Package 참고 하세요. --> [requirements.txt](../requirements.txt)

## 참고 자료
- [Building with Anthropic’s Claude 3 on Amazon Bedrock and LangChain](https://medium.com/@dminhk/building-with-anthropics-claude-3-on-amazon-bedrock-and-langchain-%EF%B8%8F-2b842f9c0ca8)
- [Amazon Bedrock 기반 Amorepacific 리뷰 요약 서비스 평가 방법 구현하기](langchain_core.runnables.base.RunnableSequence)
- [Amazon Bedrock model IDs](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html)

---

# 0. 선수 노트북
- 아래의 노트북을 먼저 실행하셔야 합니다.  --> [03_matching_image_element_template.ipynb](03_matching_image_element_template.ipynb)


# 1. 환경 셋업

가장 처음 실행시에는 아래를 install_needed == True 로 실행 해주세요

In [1]:
import sys

install_needed = True
# install_needed = False

if install_needed:
    !{sys.executable} -m pip install -q langchain==0.1.16
    !{sys.executable} -m pip install -q langchain-community==0.0.34
    !{sys.executable} -m pip install -q langchain-aws==0.1.2
    !{sys.executable} -m pip install -q boto3==1.34.87    

In [2]:
! pip list | grep -E "langchain|boto3"

boto3                                 1.34.87
langchain                             0.1.16
langchain-aws                         0.1.2
langchain-community                   0.0.34
langchain-core                        0.1.52
langchain-text-splitters              0.0.2


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)

python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/05_image_styling_recommendation_with_prompt_engineering/evaluation is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/05_image_styling_recommendation_with_prompt_engineering/evaluation/notebook', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/05_image_styling_recommendation_with_prompt_engineering/evaluation']


# 2. Ground Truth (Label) 생성하기

## 두개의 모델에서 선택한 이미지 요소에 대한 이유 가져오기
- 이전 노트북에서 저장한 두개의 모델의 이유를 가져 옵니다

In [5]:
from eval_utils.langchain_bedrock import BedrockLangChain_Image
from eval_utils.util import (
                        print_ww
                        # invoke_model_text2text,                       
)


In [6]:
%store -r select_reason_sonnet
%store -r select_reason_haiku

try:
    print("**select_reason_sonnet:")
    print_ww(select_reason_sonnet)
    print("**select_reason_haiku: ")
    print_ww(select_reason_haiku)
except:
    print("Run 03_matching_image_element_template.ipynb")




**select_reason_sonnet:
이 이미지는 대상 이미지와 가장 유사한 시각적 요소와 주제를 가지고 있습니다. 상단에 만개한 벚꽃 가지가 있어 봄의 분위기를 자아내며, 중앙의 큰 텍스트는 이벤트나 프로모션을
알리는 듯합니다. 또한 하단의 브랜드 로고와 연락처 정보가 있어 실제 상업적 용도로 활용될 수 있습니다. 전체적으로 부드럽고 밝은 분홍색 계열의 색상이 사용되어 대상 이미지와
유사한 따뜻하고 화사한 분위기를 연출합니다. 이러한 시각적 구성과 봄 여행 프로모션이라는 주제 면에서 가장 유사성이 높습니다.
**select_reason_haiku: 
candidate_image_2.jpg가 <target_image>와 가장 유사한 것으로 판단됩니다. 두 이미지 모두 봄의 분위기를 잘 표현하고 있으며, 벚꽃 이미지와 밝은 색상
사용, 텍스트 레이아웃 등 전반적인 시각적 구성이 유사합니다. 특히 <target_image>의 벚꽃 풍경과 중앙의 텍스트 박스, 화살표 등의 요소가
candidate_image_2.jpg에서도 유사하게 나타나고 있습니다. 또한 두 이미지 모두 봄 시즌 프로모션이나 이벤트를 알리는 용도로 활용될 수 있어 주제와 활용 분야에서도
일치합니다. 따라서 candidate_image_2.jpg가 <target_image>와 가장 유사한 템플릿이라고 판단됩니다.


## Ground Truth 생성을 위한 System, User Prompt 확인하기
- Ground Truth 를 사람이 작성하는 것을 권장하나, 너무 많은 노력이 들어 갈 수 있어서 여기서는 아래와 같이 가정하여 작성합니다.
    - Claude3 Sonnet 모델이 어울리다가 생각한 "이유" 를 Claude3 Sonnet 모델에 Re-Write 한 것을 Ground Truth 가정 합니다.
- 아래의 System Prompt & User Prompt 는 미리 Claude3 Sonnet 모델을 이용해서 작성한 것을 가져옵니다.

In [22]:
import boto3
from eval_utils.prompt import ImagePrompt

FP = ImagePrompt()
system_prompt = FP.get_rewrite_system_prompt()
user_prompt = FP.get_rewrite_user_prompt()

print("## system_prompt")
print_ww(system_prompt)
print("## user_prompt")
print_ww(user_prompt)


## system_prompt
The task is to rewrite a given sentence in a different way while preserving its original
meaning.Your role is to take a sentence provided by the user and rephrase it using different words
or sentence structures, without altering the core meaning or message conveyed in the original
sentence.

Instructions:
1. Read the sentence carefully and ensure you understand its intended meaning.
2. Identify the key components of the sentence, such as the subject, verb, object, and any modifiers
or additional information.
3. Think of alternative ways to express the same idea using different vocabulary, sentence
structures, or phrasing.
4. Ensure that your rewritten sentence maintains the same essential meaning as the original, without
introducing any new information or altering the original intent.
5. Pay attention to grammar, punctuation, and overall coherence to ensure your rewritten sentence is
well-formed and easy to understand.
6. If the original sentence contains idioms, metap

## Sonnet 모델의 Ground Truth 를 Sonnet 모델을 이용하여 Rw-Write 하여 생성

In [23]:

region = "us-west-2"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

client = boto3.client(service_name="bedrock-runtime", region_name=region)
model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 1.0,
    "top_k": 250,
    "top_p": 1,
}                             

BL = BedrockLangChain_Image(bedrock_runtime=client)
response = BL.invoke_rewrite_langchain(model_id = model_id, 
                             model_kwargs = model_kwargs, 
                             user_prompt = user_prompt, 
                             system_prompt = system_prompt, 
                             recommendation_review = select_reason_sonnet,
                             verbose=True)

import json
data_dict = json.loads(response)
rewrite_original_recommendation_review = data_dict['rewrite_original_recommendation_review']
print("## Re-Write recommendation_review ")
print_ww(rewrite_original_recommendation_review)

messages: 
 [('system', 'The task is to rewrite a given sentence in a different way while preserving its original meaning.Your role is to take a sentence provided by the user and rephrase it using different words or sentence structures, without altering the core meaning or message conveyed in the original sentence.\n\nInstructions:\n1. Read the sentence carefully and ensure you understand its intended meaning.\n2. Identify the key components of the sentence, such as the subject, verb, object, and any modifiers or additional information.\n3. Think of alternative ways to express the same idea using different vocabulary, sentence structures, or phrasing.\n4. Ensure that your rewritten sentence maintains the same essential meaning as the original, without introducing any new information or altering the original intent.\n5. Pay attention to grammar, punctuation, and overall coherence to ensure your rewritten sentence is well-formed and easy to understand.\n6. If the original sentence contai

# 4. Evaluation 기준 및 방법을 생성하는 프롬프트 만들기
- "Evaluation 기준 및 방법" 을 여러가지 솔루션으로 구현할 수 있습니다. 여기서는 Claude3 Sonnet 에게 이런한 기준을 작성하게 했습니다.
    - 아래는 이러한 기준을 작성할 수 있다는 예시 입니다. 
    - 실제로 아래 프롬프트를 사용하여 생성을 받았고, 이후에 약간의 편집을 통해서 ../eval_utils/prompt.py 에 저장을 했습니다. 
    - 여기 셀의 실행은 이렇게 할 수 있다는 것만을 보여 드립니다.


## Evaluation 기준 및 방법을 생성하는 프롬프트



In [37]:
from eval_utils.prompt import ImagePrompt

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
}                             


FP = ImagePrompt()
system_prompt = FP.get_create_criteria_system_prompt()
user_prompt = FP.get_create_criteria_user_prompt()

guide="이후에 당신은 이미지 템플릿과 어울리는 이미지 요소 추천에 대해 그래픽 디자이너의 의견과 AI가 제공한 두가지 의견이 주어질 겁니다.  \
이 두개의 의견이 얼마나 연관성이 있는지를 1점, 2점, 3점, 4점, 5점 으로서  스코어의 기준을 <criterira></criterira>  형식으로 작성하시고, \
두개의 의견이 제공이 되었을때에 평가를 하는 단계를 <steps></steps> 형식으로 작성 하세요. "


BL = BedrockLangChain_Image(bedrock_runtime=client)
BL.invoke_creating_criteria_langchain(model_id = model_id, 
                             model_kwargs = model_kwargs, 
                             user_prompt = user_prompt, 
                             system_prompt = system_prompt, 
                             guide = guide,
                             verbose=True)


messages: 
 [('system', 'You are a prompt engineering expert.'), ('human', '먼저 당신의 역할과 작업을 XML Tag 없이 기술하세요, 이후에 아래의 <guide> 에 맟주어서 프롬프트를 영어로 작성해주세요. \n<guide>{guide}</guide>')]
prompt: 

input_variables=['guide']
messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a
prompt engineering expert.')),
HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['guide'], template='먼저 당신의 역할과 작업을
XML Tag 없이 기술하세요, 이후에 아래의 <guide> 에 맟주어서 프롬프트를 영어로 작성해주세요. \n<guide>{guide}</guide>'))]
## Created Prompt:



As a prompt engineering expert, my role is to provide clear and effective prompts to guide AI language models in generating relevant and coherent responses. I aim to structure prompts in a way that aligns with the desired task or objective, while considering the capabilities and limitations of the AI model.

<guide>In this scenario, you will be provided with an image template and two sets of recommendations: one from a graphic designer's perspective and another from an AI's perspective, regarding suitable image elements to complement the template. Your task is to evaluate the relevance and coherence between these two recommendations.

<criteria>
1 - The recommendations have minimal or no relevance to each other.
2 - The recommendations have some relevance, but with significant differences or contradictions.
3 - The recommendations share moderate relevance, with some overlapping or complementary aspects.
4 - The recommendations are largely relevant and coherent, with only minor differen

# 5. Ground Truth 와 AI Review 평가하기

## Claude3 Sonnet 이 제공한 이유 평가하기
- ground_truth 와 Sonnet 이 작성한 이유를 비교해서 "관련성" 을 평가 합니다.

In [1]:
system_prompt = FP.get_image_recommendation_system_prompt()
user_prompt = FP.get_image_recommendation_user_prompt()


ground_truth = rewrite_original_recommendation_review


# human_message = "<target_image>의 메시지인 젊음, 활력, 자유를 상징하는 전기 기타 실루엣 이미지는 심플하고 단순한 디자인으로 <target_image>의 미니멀한 스타일과 잘 어울립니다. 또한 음악과 청춘 문화를 대표하는 요소이기에 <target_image>의 키워드인 '청춘'과도 연관성이 높습니다. 그러므로 이 이미지는 <target_image>의 콘셉트와 미학을 잘 보완해 줄 수 있을 것입니다."                  
# AI_message = "전기 기타 실루엣 이미지는 청춘과 활력, 자유로움을 상징하는 <target_image>의 메시지와 잘 어울립니다. 단순하고 심플한 디자인으로 <target_image>의 미니멀한 스타일과도 조화를 이룹니다. 또한 음악과 젊은 문화를 대표하는 요소로서 <target_image>의 '청춘'이라는 키워드와 연관성이 높습니다. 따라서 이 이미지는 <target_image>의 컨셉과 미학적 특성을 잘 보완할 수 있을 것으로 판단됩니다."                  


human_message = ground_truth
AI_message = select_reason_sonnet


BL = BedrockLangChain_Image(bedrock_runtime=client)
BL.invoke_evaluating_recommendation_review_langchain(model_id = model_id, 
                             model_kwargs = model_kwargs, 
                             user_prompt = user_prompt, 
                             system_prompt = system_prompt, 
                             AI_message = AI_message,
                             human_message = human_message,
                             verbose=False)


NameError: name 'FP' is not defined

## Claude3 Haiku가 제공한 이유 평가하기
- ground_truth 와 Haiku 가 작성한 이유를 비교해서 "관련성" 을 평가 합니다.

In [40]:
system_prompt = FP.get_image_recommendation_system_prompt()
user_prompt = FP.get_image_recommendation_user_prompt()

human_message = ground_truth
AI_message = select_reason_haiku


BL = BedrockLangChain_Image(bedrock_runtime=client)
BL.invoke_evaluating_recommendation_review_langchain(model_id = model_id, 
                             model_kwargs = model_kwargs, 
                             user_prompt = user_prompt, 
                             system_prompt = system_prompt, 
                             AI_message = AI_message,
                             human_message = human_message,
                             verbose=False)



<evaluation>
{
"human_view": "<target_image>의 메시지인 젊음, 활력, 자유를 상징하는 전기 기타 실루엣 이미지는 심플하고 단순한 디자인으로 <target_image>의 미니멀한 스타일과 잘 어울립니다. 또한 음악과 청춘 문화를 대표하는 요소이기에 <target_image>의 키워드인 '청춘'과도 연관성이 높습니다. 그러므로 이 이미지는 <target_image>의 콘셉트와 미학을 잘 보완해 줄 수 있을 것입니다.",
"AI_view": "이 전기 기타 실루엣 이미지는 <target_image>의 주요 시각 요소와 잘 어울립니다. 두 이미지 모두 단순하고 심플한 스타일로 구성되어 있어 전체적인 미적 일관성을 유지할 수 있습니다. 또한 전기 기타는 젊음과 활력을 상징하는 악기로, <target_image>에서 강조하는 '청춘'이라는 메시지와도 잘 부합합니다. 이 이미지는 다양한 매체에 활용할 수 있는 범용성과 확장성을 가지고 있어, 마케팅 자료 및 브랜딩 등에 효과적으로 사용될 수 있을 것으로 판단됩니다.",
"score": 4,
"reason": "두 의견 모두 전기 기타 실루엣 이미지가 <target_image>의 미니멀한 스타일과 잘 어울리며, 젊음과 활력을 상징하는 '청춘' 메시지와도 연관성이 높다는 점에서 일치합니다. 다만 AI 의견에서는 이미지의 범용성과 확장성에 대해 추가적으로 언급하고 있습니다."
}
</evaluation>

## 실험으로 관련성이 적은 리뷰 제공
- 앞 실험에서는 Sonnet, Haiku 가 비슷한 스코어를 (예: 4점) 을 주었기 때문에, AI_review로 다른 것을 입력으로 하여, 스코어와 이유를 비교 합니다.

In [41]:
ground_truth = rewrite_original_coordination_review


human_message = ground_truth
AI_message = "열대어 요소 이미지가 <target image>에 어울립니다. 배경 이미지는 바다이므로, 열대어가 어울리며, 열대어의 노란색이 배경 이미지 바다색이 대비되어 요소를 강조할 수 있습니다."                  


BL = BedrockLangChain_Image(bedrock_runtime=client)
BL.invoke_evaluating_recommendation_review_langchain(model_id = model_id, 
                             model_kwargs = model_kwargs, 
                             user_prompt = user_prompt, 
                             system_prompt = system_prompt, 
                             AI_message = AI_message,
                             human_message = human_message,
                             verbose=False)


<evaluation>
{
"human_view": "<target_image>의 메시지인 젊음, 활력, 자유를 상징하는 전기 기타 실루엣 이미지는 심플하고 단순한 디자인으로 <target_image>의 미니멀한 스타일과 잘 어울립니다. 또한 음악과 청춘 문화를 대표하는 요소이기에 <target_image>의 키워드인 '청춘'과도 연관성이 높습니다. 그러므로 이 이미지는 <target_image>의 콘셉트와 미학을 잘 보완해 줄 수 있을 것입니다.",
"AI_view": "열대어 요소 이미지가 <target image>에 어울립니다. 배경 이미지는 바다이므로, 열대어가 어울리며, 열대어의 노란색이 배경 이미지 바다색이 대비되어 요소를 강조할 수 있습니다.",
"score": 2,
"reason": "두 의견은 서로 다른 이미지 요소를 제안하고 있으며, 전반적인 주제와 권장 사항이 크게 다릅니다."
}
</evaluation>